In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import date


# Storage



In [ ]:
sitemap_url = "https://www.uscc.gov/sitemap.xml"
response = requests.get(sitemap_url)
soup = BeautifulSoup(response.text, "xml")

# Extract all URLs
all_urls = [loc.text for loc in soup.find_all("loc")]
hearing_urls = sorted(set([u for u in all_urls if "/hearings/" in u]))

print("Total hearings collected:", len(hearing_urls))
print("Earliest hearing:", hearing_urls[0])
print("Latest hearing:", hearing_urls[-1])

# Helpers

Total hearings collected: 180
Earliest hearing: https://www.uscc.gov/hearings/assessment-ccps-economic-ambitions-plans-and-metrics-success
Latest hearing: https://www.uscc.gov/hearings/world-class-military-assessing-chinas-global-military-ambitions


In [ ]:

def parse_hearing(url):
    r = requests.get(url)
    page = BeautifulSoup(r.text, "lxml")

    # Title
    title = page.find("h1").get_text(strip=True) if page.find("h1") else ""

    # Date (prefer structured element)
    date_tag = page.find("span", class_="date-display-single")
    if date_tag:
        date = date_tag.get_text(strip=True)
    else:
        # fallback regex
        text = page.get_text(" ", strip=True)
        m = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}", text)
        date = m.group(0) if m else ""

    # Transcript(s)
    transcripts = []
    for a in page.find_all("a", href=True):
        if "Transcript" in a.text or "Transcript" in a["href"]:
            link = "https://www.uscc.gov" + a["href"] if a["href"].startswith("/") else a["href"]
            transcripts.append(link)
    transcript_link = "; ".join(transcripts) if transcripts else None

    # Speakers
    speakers = []

    # Case 1: Bio PDFs
    for a in page.find_all("a", href=True):
        if a["href"].endswith("_Bio.pdf"):
            name = a["href"].split("/")[-1].replace("_Bio.pdf", "").replace("_", " ")
            link = "https://www.uscc.gov" + a["href"] if a["href"].startswith("/") else a["href"]
            speakers.append({"hearing_url": url, "name": name, "profile_url": link})

    # Case 2: Witness sections (catch plain text names too)
    for div in page.select(".field--name-field-witness .field__item"):
        name = div.get_text(" ", strip=True)
        if name and not any(s["name"] == name for s in speakers):
            speakers.append({"hearing_url": url, "name": name, "profile_url": None})

    return {
        "url": url,
        "title": title,
        "date": date,
        "transcript": transcript_link,
        "speakers": speakers
    }


In [ ]:
hearing_data, speaker_data = [], []

for i, u in enumerate(hearing_urls, 1):
    try:
        parsed = parse_hearing(u)

        hearing_data.append({
            "url": parsed["url"],
            "title": parsed["title"],
            "date": parsed["date"],
            "transcript": parsed["transcript"]
        })

        speaker_data.extend(parsed["speakers"])

    except Exception as e:
        print(f"Error at {u}: {e}")

    # Polite delay
    time.sleep(1)

    # Checkpoint
    if i % 25 == 0:
        pd.DataFrame(hearing_data).to_csv("checkpoint_hearings.csv", index=False)
        pd.DataFrame(speaker_data).to_csv("checkpoint_speakers.csv", index=False)
        print(f"Checkpoint saved at {i} hearings")

# Final save with today’s date
today = date.today().strftime("%Y%m%d")
df_master = pd.DataFrame(hearing_data)
df_speakers = pd.DataFrame(speaker_data)

df_master.to_csv(f"{today}_uscc_master_hearings.csv", index=False)
df_speakers.to_csv(f"{today}_uscc_master_speakers.csv", index=False)

print("Scraping complete ✅")

Checkpoint saved at 25 hearings
Checkpoint saved at 50 hearings
Checkpoint saved at 75 hearings
Checkpoint saved at 100 hearings
Checkpoint saved at 125 hearings
Checkpoint saved at 150 hearings
Checkpoint saved at 175 hearings
Scraping complete ✅


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.uscc.gov/hearings/assessment-ccps-economic-ambitions-plans-and-metrics-success"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

speakers = []
for block in soup.select(".views-row"):
    name = block.select_one(".views-field-title").get_text(strip=True) if block.select_one(".views-field-title") else None
    affiliation = block.select_one(".views-field-field-speaker-affiliation").get_text(strip=True) if block.select_one(".views-field-field-speaker-affiliation") else None
    speakers.append((name, affiliation))

print(speakers)


[]


In [ ]:
print(soup.prettify()[:2000])  # show first 2000 chars


<!--

                            _                       _     _
                           | |                     | |   (_)
      _ __   _____      __ | |_ __ _ _ __ __ _  ___| |_   _ _ __   ___
     | '_ \ / _ \ \ /\ / / | __/ _` | '__/ _` |/ _ \ __| | | '_ \ / __|
     | | | |  __/\ V  V /  | || (_| | | | (_| |  __/ |_  | | | | | (__ _
     |_| |_|\___| \_/\_/    \__\__,_|_|  \__, |\___|\__| |_|_| |_|\___(_)
                                          __/ |
                                         |___/

     We create digital marketing strategies and design compelling websites.
                            www.newtarget.com

-->
<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/0

In [ ]:
import requests
import pdfplumber
from io import BytesIO

url = "https://www.uscc.gov/sites/default/files/2021-04/Matt_Pottinger_Testimony.pdf"
response = requests.get(url)

with pdfplumber.open(BytesIO(response.content)) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text() + "\n"

print(text[:1000])  # preview first 1000 chars


ModuleNotFoundError: No module named 'pdfplumber'

In [ ]:
!pip install pdfplumber



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.5 MB/s eta 0:00:00


In [ ]:
import pdfplumber


In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.9 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import requests
from io import BytesIO

url = "https://www.uscc.gov/sites/default/files/2021-04/Matt_Pottinger_Testimony.pdf"
response = requests.get(url)

doc = fitz.open(stream=BytesIO(response.content), filetype="pdf")
text = ""
for page in doc:
    text += page.get_text()

print(text[:1000])  # preview first 1000 chars


1 
 
15 April 2021 
 
Statement of Matt Pottinger 
 
Distinguished Visiting Fellow at the Hoover Institution, 
Stanford University 
Former Assistant to the President and Deputy National 
Security Advisor, the White House 
 
Testimony Before the United States-China Economic and 
Security Review Commission  
Chairman Bartholomew, Vice Chairman Dr. Cleveland, and all 
Commissioners, thank you for the opportunity to speak about the 
Chinese Communist Party’s economic strategy, and to touch on 
a few principles that I believe the United States and other free 
nations should apply as we hone our collective counter-strategy.   
The Communist regime’s ambitions really shouldn’t be a 
mystery to us anymore.  If we listen to what China’s leaders 
have been saying in their own language, to their own Party 
members, and cross-reference their rhetoric with their actions, 
we can see that their plans are hiding in plain sight.   
China’s latest Five-Year Plan, published last month, 
institutionalize

In [ ]:
lines = text.splitlines()



In [ ]:
for i, line in enumerate(lines):
    if line.strip().startswith("Statement of"):
        name = line.replace("Statement of", "").strip()
        affiliation = " ".join(lines[i+1:i+3]).strip()  # take next 1–2 lines
        description = " ".join(lines[i+3:]).strip()
        break


In [ ]:
import fitz  # PyMuPDF
import requests
from io import BytesIO
import pandas as pd

# Load your speakers master file
speakers_df = pd.read_csv("20250929_uscc_master_speakers.csv")

# New columns
speakers_df["affiliation"] = ""
speakers_df["description"] = ""

def extract_info_from_pdf(pdf_url):
    try:
        response = requests.get(pdf_url, timeout=20)
        response.raise_for_status()
        doc = fitz.open(stream=BytesIO(response.content), filetype="pdf")

        text = ""
        for page in doc:
            text += page.get_text()

        lines = [l.strip() for l in text.splitlines() if l.strip()]

        name, affiliation, description = None, None, None

        for i, line in enumerate(lines):
            if line.startswith(("Statement of", "Testimony of")):
                name = line.split("of",1)[-1].strip()
                affiliation = " ".join(lines[i+1:i+3])  # next 1–2 lines
                description = " ".join(lines[i+3:])
                break

        return name, affiliation, description

    except Exception as e:
        print(f"Error processing {pdf_url}: {e}")
        return None, None, None

# Loop through all speakers
for idx, row in speakers_df.iterrows():
    pdf_url = row["profile_url"]
    name, affiliation, description = extract_info_from_pdf(pdf_url)

    if name:  # only overwrite if found
        speakers_df.at[idx, "name"] = name
    speakers_df.at[idx, "affiliation"] = affiliation
    speakers_df.at[idx, "description"] = description

# Save updated file
speakers_df.to_csv("20250929_uscc_master_speakers_cleaned.csv", index=False)


Error processing https://www.uscc.gov/sites/default/files/Panel%20III_Swanstr%C3%B6m_Bio.pdf: 404 Client Error: Not Found for url: https://www.uscc.gov/sites/default/files/Panel%20III_Swanstr%C3%B6m_Bio.pdf


In [ ]:
for idx, row in speakers_df.iterrows():
    pdf_url = row["profile_url"]

    try:
        name, affiliation, description = extract_info_from_pdf(pdf_url)

        if name:  # only overwrite if found
            speakers_df.at[idx, "name"] = name
        speakers_df.at[idx, "affiliation"] = affiliation
        speakers_df.at[idx, "description"] = description
        speakers_df.at[idx, "bio_status"] = "ok"

    except Exception as e:
        print(f"Skipping {pdf_url}: {e}")
        speakers_df.at[idx, "bio_status"] = "missing_pdf"


Error processing https://www.uscc.gov/sites/default/files/Panel%20III_Swanstr%C3%B6m_Bio.pdf: 404 Client Error: Not Found for url: https://www.uscc.gov/sites/default/files/Panel%20III_Swanstr%C3%B6m_Bio.pdf


In [ ]:
speakers_df.to_csv("20250929_uscc_master_speakers_cleaned.csv", index=False)


In [ ]:
import fitz  # PyMuPDF
import requests
from io import BytesIO

# Test with one speaker's PDF
pdf_url = "https://www.uscc.gov/sites/default/files/2021-04/Matt_Pottinger_Testimony.pdf"

response = requests.get(pdf_url)
doc = fitz.open(stream=BytesIO(response.content), filetype="pdf")

# Extract text
text = ""
for page in doc:
    text += page.get_text()

lines = [l.strip() for l in text.splitlines() if l.strip()]

name, affiliation, description = None, None, None

for i, line in enumerate(lines):
    if line.startswith(("Statement of", "Testimony of")):
        name = line.split("of", 1)[-1].strip()
        affiliation = " ".join(lines[i+1:i+3])
        description = " ".join(lines[i+3:i+10])  # first few lines of bio/testimony
        break

print("Name:", name)
print("Affiliation:", affiliation)
print("Description (preview):", description[:300], "...")


Name: Matt Pottinger
Affiliation: Distinguished Visiting Fellow at the Hoover Institution, Stanford University
Description (preview): Former Assistant to the President and Deputy National Security Advisor, the White House Testimony Before the United States-China Economic and Security Review Commission Chairman Bartholomew, Vice Chairman Dr. Cleveland, and all Commissioners, thank you for the opportunity to speak about the Chinese  ...


In [ ]:
def extract_info_from_pdf(pdf_url):
    try:
        response = requests.get(pdf_url, timeout=20)
        response.raise_for_status()
        doc = fitz.open(stream=BytesIO(response.content), filetype="pdf")

        text = ""
        for page in doc:
            text += page.get_text()

        lines = [l.strip() for l in text.splitlines() if l.strip()]

        name, affiliation, description = None, None, None

        for i, line in enumerate(lines):
            if line.startswith(("Statement of", "Testimony of")):
                name = line.split("of", 1)[-1].strip()
                # Sometimes affiliation spans 2–3 lines
                affiliation = " ".join(lines[i+1:i+4])
                # Keep first 8–10 lines as description (bio snippet)
                description = " ".join(lines[i+4:i+12])
                break

        return name, affiliation, description

    except Exception as e:
        print(f"Error processing {pdf_url}: {e}")
        return None, None, None


In [ ]:
import fitz  # PyMuPDF
import requests
from io import BytesIO
import pandas as pd

# Load the full speakers file
speakers_df = pd.read_csv("20250929_uscc_master_speakers.csv")

def extract_info_from_pdf(pdf_url):
    try:
        response = requests.get(pdf_url, timeout=20)
        response.raise_for_status()
        doc = fitz.open(stream=BytesIO(response.content), filetype="pdf")

        text = ""
        for page in doc:
            text += page.get_text()

        lines = [l.strip() for l in text.splitlines() if l.strip()]

        name, affiliation, description = None, None, None

        if "Bio.pdf" in pdf_url:  # handle bios
            name = pdf_url.split("/")[-1].replace("_Bio.pdf", "").replace("_", " ")
            affiliation = lines[0] if lines else None
            description = " ".join(lines[1:]) if len(lines) > 1 else None
        else:  # handle testimonies
            keywords = ["Statement of", "Testimony of", "Prepared Statement of", "Biography of"]
            for i, line in enumerate(lines):
                if any(line.startswith(k) for k in keywords):
                    name = line.split("of", 1)[-1].strip()
                    affiliation = " ".join(lines[i+1:i+4])
                    description = " ".join(lines[i+4:i+12])
                    break

        return name, affiliation, description

    except Exception as e:
        print(f"Error processing {pdf_url}: {e}")
        return None, None, None


# --- Process all speakers ---
for idx, row in speakers_df.iterrows():
    pdf_url = row["profile_url"]

    name, affiliation, description = extract_info_from_pdf(pdf_url)

    if name:
        speakers_df.at[idx, "name"] = name
    if affiliation:
        speakers_df.at[idx, "affiliation"] = affiliation
    if description:
        speakers_df.at[idx, "description"] = description

    # progress log every 25 speakers
    if idx % 25 == 0:
        print(f"Processed {idx+1}/{len(speakers_df)} -> {name}")

# --- Save enriched dataset ---
speakers_df.to_csv("uscc_master_speakers_enriched.csv", index=False)

print("\nDone ✅")
print("Saved as: uscc_master_speakers_enriched.csv")
print(speakers_df[["name", "affiliation", "description"]].head())


Processed 1/431 -> Matt Pottinger
Processed 26/431 -> Nis Gr%C3%BCnberg
Processed 51/431 -> Timothy Meyer
Processed 76/431 -> Tayyab Safdar
Processed 101/431 -> John Chen
Processed 126/431 -> J Michael Dahm
Processed 151/431 -> Jeffrey Becker
Processed 176/431 -> Katja Drinhausen
Processed 201/431 -> Jim Joholske
Processed 226/431 -> Ngor Luong
Processed 251/431 -> Shichor
Processed 276/431 -> Prasad%2C%20Eswar
Processed 301/431 -> Panel%202 Wang
Error processing https://www.uscc.gov/sites/default/files/Panel%20III_Swanstr%C3%B6m_Bio.pdf: 404 Client Error: Not Found for url: https://www.uscc.gov/sites/default/files/Panel%20III_Swanstr%C3%B6m_Bio.pdf
Processed 326/431 -> David%20Wertime
Processed 351/431 -> Julia Friedlander
Processed 376/431 -> Yu-Jie Chen
Processed 401/431 -> Maureen Thorson
Processed 426/431 -> Panel%20II Michael%20Hirson

Done ✅
Saved as: uscc_master_speakers_enriched.csv
              name          affiliation  \
0   Matt Pottinger       Matt Pottinger   
1        

In [ ]:
import pandas as pd

df = pd.read_csv("uscc_master_speakers_enriched.csv")

def clean_affiliation(row):
    name = row["name"]
    aff = str(row["affiliation"]) if pd.notnull(row["affiliation"]) else ""
    desc = str(row["description"]) if pd.notnull(row["description"]) else ""

    # 1. If affiliation is just the name, replace with first line of description
    if aff.strip() == name.strip() or aff.strip().startswith(name.strip()):
        first_line = desc.split(".")[0] if desc else ""
        return first_line

    # 2. Remove trailing commas/semicolons
    aff = aff.strip(" ,;")

    # 3. If affiliation is very long, shorten it to first sentence
    if len(aff.split(".")) > 1:
        aff = aff.split(".")[0]

    return aff

df["affiliation"] = df.apply(clean_affiliation, axis=1)

# Save cleaned version
df.to_csv("uscc_master_speakers_cleaned_final.csv", index=False)

print("✅ Cleaned file saved as uscc_master_speakers_cleaned_final.csv")
print(df.head(5)[["name", "affiliation", "description"]])


✅ Cleaned file saved as uscc_master_speakers_cleaned_final.csv
              name                                        affiliation  \
0   Matt Pottinger  Distinguished Visiting Fellow, Hoover Institut...   
1         Miles Yu  Senior Fellow, Hudson Institute; Visiting Fell...   
2     Loren Brandt  Noranda Chair Professor of Economics, Universi...   
3  Jude Blanchette  Freeman Chair in China Studies, Center for Str...   
4        Ling Chen  Assistant Professor of Political Economy, Scho...   

                                         description  
0  Distinguished Visiting Fellow, Hoover Institut...  
1  Senior Fellow, Hudson Institute; Visiting Fell...  
2  Noranda Chair Professor of Economics, Universi...  
3  Freeman Chair in China Studies, Center for Str...  
4  Assistant Professor of Political Economy, Scho...  


In [ ]:
import pandas as pd

# Load your hearings file
hearings_df = pd.read_csv("20250929_uscc_master_hearings.csv")

# Filter for missing dates or transcripts
missing_rows = hearings_df[hearings_df["date"].isna() | hearings_df["transcript"].isna()]

print(missing_rows[["url", "title", "date", "transcript"]])


                                                   url  \
42   https://www.uscc.gov/hearings/field-investigat...   
47   https://www.uscc.gov/hearings/hearing-bilatera...   
50   https://www.uscc.gov/hearings/hearing-china-an...   
52   https://www.uscc.gov/hearings/hearing-china-an...   
54   https://www.uscc.gov/hearings/hearing-china-an...   
55   https://www.uscc.gov/hearings/hearing-china-an...   
56   https://www.uscc.gov/hearings/hearing-china-an...   
57   https://www.uscc.gov/hearings/hearing-china-em...   
60   https://www.uscc.gov/hearings/hearing-china-tr...   
64   https://www.uscc.gov/hearings/hearing-chinas-a...   
68   https://www.uscc.gov/hearings/hearing-chinas-e...   
72   https://www.uscc.gov/hearings/hearing-chinas-f...   
77   https://www.uscc.gov/hearings/hearing-chinas-g...   
78   https://www.uscc.gov/hearings/hearing-chinas-g...   
80   https://www.uscc.gov/hearings/hearing-chinas-h...   
86   https://www.uscc.gov/hearings/hearing-chinas-i...   
88   https://w

In [ ]:
import requests
from bs4 import BeautifulSoup

test_url = missing_rows.iloc[0]["url"]  # take the first missing one
response = requests.get(test_url)
soup = BeautifulSoup(response.text, "html.parser")

# Get date
date = soup.select_one(".field--name-field-hearing-date")
date = date.get_text(strip=True) if date else None

# Get transcript link (PDF)
transcript_link = None
for a in soup.find_all("a", href=True):
    if a["href"].endswith(".pdf") and "Transcript" in a.text:
        transcript_link = "https://www.uscc.gov" + a["href"]
        break

print("Date:", date)
print("Transcript:", transcript_link)


Date: None
Transcript: None


In [ ]:
for idx, row in hearings_df.iterrows():
    if pd.isna(row["date"]) or pd.isna(row["transcript"]):
        response = requests.get(row["url"])
        soup = BeautifulSoup(response.text, "html.parser")

        # Date
        date = soup.select_one(".field--name-field-hearing-date")
        if date:
            hearings_df.at[idx, "date"] = date.get_text(strip=True)

        # Transcript
        transcript_link = None
        for a in soup.find_all("a", href=True):
            if a["href"].endswith(".pdf") and "Transcript" in a.text:
                transcript_link = "https://www.uscc.gov" + a["href"]
                break
        if transcript_link:
            hearings_df.at[idx, "transcript"] = transcript_link


In [ ]:
hearings_df.to_csv("uscc_master_hearings_cleaned.csv", index=False)


In [ ]:
import pandas as pd

hearings_cleaned_df = pd.read_csv("uscc_master_hearings_cleaned.csv")


In [ ]:
missing_rows = hearings_cleaned_df[
    hearings_cleaned_df["date"].isna() | hearings_cleaned_df["transcript"].isna()
]
print(missing_rows.head(3)[["url", "title", "date", "transcript"]])



                                                  url  \
42  https://www.uscc.gov/hearings/field-investigat...   
47  https://www.uscc.gov/hearings/hearing-bilatera...   
50  https://www.uscc.gov/hearings/hearing-china-an...   

                                                title              date  \
42  Field Investigation on China's Impact on the U...  January 30, 2004   
47  Hearing: Bilateral Trade Policies and Issues B...    August 2, 2001   
50             Hearing: China and the Capital Markets   August 11, 2005   

   transcript  
42        NaN  
47        NaN  
50        NaN  


In [ ]:
import re

def clean_name(name, affiliation):
    # Remove artifacts like "Panel%203"
    name = re.sub(r"Panel.*", "", name, flags=re.IGNORECASE).strip()

    # Remove academic suffixes
    name = re.sub(r",?\s*(Ph\.D\.|M\.D\.|J\.D\.)", "", name, flags=re.IGNORECASE).strip()

    # Fix spacing/capitalization
    name = " ".join(name.split())

    # Add honorifics (optional)
    if "Professor" in str(affiliation) or "Doctor" in str(affiliation):
        name = "Dr. " + name
    elif name and not name.startswith(("Mr.", "Ms.", "Dr.")):
        name = "Mr. " + name

    return name


In [ ]:
speakers_df["name_clean"] = speakers_df.apply(
    lambda row: clean_name(row["name"], row["affiliation"]), axis=1
)

# Preview
print(speakers_df[["name", "name_clean"]].head(10))


               name            name_clean
0    Matt Pottinger    Mr. Matt Pottinger
1          Miles Yu          Mr. Miles Yu
2      Loren Brandt      Mr. Loren Brandt
3   Jude Blanchette   Mr. Jude Blanchette
4         Ling Chen         Mr. Ling Chen
5        Nigel Cory        Mr. Nigel Cory
6       Jason Kelly       Mr. Jason Kelly
7      Joanna Moody      Mr. Joanna Moody
8  Martin Chorzempa  Mr. Martin Chorzempa
9      Yaya Fanusie      Mr. Yaya Fanusie


In [ ]:
import re

def clean_name(name, affiliation):
    # Remove artifacts like Panel%20
    name = re.sub(r"Panel.*", "", str(name), flags=re.IGNORECASE).strip()
    # Remove academic suffixes (Ph.D., M.D., J.D.)
    name = re.sub(r",?\s*(Ph\.D\.|M\.D\.|J\.D\.)", "", name, flags=re.IGNORECASE).strip()
    # Fix spacing
    name = " ".join(name.split())

    # Add Dr. only if affiliation suggests it
    if "Professor" in str(affiliation) or "Doctor" in str(affiliation):
        if not name.startswith("Dr. "):
            name = "Dr. " + name

    return name

# Apply cleanup
speakers_df["name_clean"] = speakers_df.apply(
    lambda row: clean_name(row["name"], row["affiliation"]), axis=1
)

# Preview
print(speakers_df[["name", "affiliation", "name_clean"]].head(15))


                     name                affiliation             name_clean
0          Matt Pottinger             Matt Pottinger         Matt Pottinger
1                Miles Yu            Miles Yu, Ph.D.               Miles Yu
2            Loren Brandt        Loren Brandt, Ph.D.           Loren Brandt
3         Jude Blanchette            Jude Blanchette        Jude Blanchette
4               Ling Chen           Ling Chen, Ph.D.              Ling Chen
5              Nigel Cory                 Nigel Cory             Nigel Cory
6             Jason Kelly         Jason Kelly, Ph.D.            Jason Kelly
7            Joanna Moody        Joanna Moody, Ph.D.           Joanna Moody
8        Martin Chorzempa           Martin Chorzempa       Martin Chorzempa
9            Yaya Fanusie               Yaya Fanusie           Yaya Fanusie
10            Sam Hoffman    Samantha Hoffman, Ph.D.            Sam Hoffman
11  Andrea Kendall-Taylor  Dr. Andrea Kendall-Taylor  Andrea Kendall-Taylor
12     Chris